In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

## Load test dataset

In [2]:
dotenv_path = "../.env"
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

In [ ]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

,Unnamed: 0.1,writer,problem_id,submission_id,website,source,Unnamed: 0
0,0,AI,164A,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN
1,1,AI,NaN,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2,2,AI,o61_may08_estate,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,1843.0
3,3,human,NaN,13646458.0,https://codeforces.com,//Template\n\n// By Anudeep :)\n//Includes\n#i...,NaN
4,4,AI,1032,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,998.0
...,...,...,...,...,...,...,...
2995,2995,human,1000,306681.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2996,2996,human,1055,310229.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2997,2997,Human,NaN,NaN,https://firefly.gchan.moe,#include <bits/stdc++.h>\n\nusing namespace st...,NaN
2998,2998,AI,888F,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN


In [4]:
# Prompt URL
prompt_path = "./Prompts/PromptV1.txt"

with open(prompt_path,'r') as _prompt:
  prompt = _prompt.read()
prompt

'You are the best C++ code detector. Your job is to determine whether the competitive programming source code is AI-generated or written by a human.\n\n### Steps:\n1. **Read the source code carefully** to understand what does the code doing.\n2. **Identify the patterns of code** and analyze whter it\'s AI-generated or not, like checking unnesscary space, variables, new lines, or comments.\n3. **Determine if it\'s Human or AI** from the data you have.\n4. **From the reason** Answer if it AI or Human.\n\nYou must respond in JSON format, like this:\n```json\n{   \n    "reason": "Why this code is human-written or AI-generated"\n    "answer": "human" / "AI",  \n}\n```\n- You must not provide any other text, except JSON.\n- In the reason part, don\'t use symbol like " , ` , \\n, use only normal character.\nHere is C++ source code:\n'

In [5]:
test_df['prompt'] = prompt + "\n\n" + test_df['source']
test_df['prompt'][0]

'You are the best C++ code detector. Your job is to determine whether the competitive programming source code is AI-generated or written by a human.\n\n### Steps:\n1. **Read the source code carefully** to understand what does the code doing.\n2. **Identify the patterns of code** and analyze whter it\'s AI-generated or not, like checking unnesscary space, variables, new lines, or comments.\n3. **Determine if it\'s Human or AI** from the data you have.\n4. **From the reason** Answer if it AI or Human.\n\nYou must respond in JSON format, like this:\n```json\n{   \n    "reason": "Why this code is human-written or AI-generated"\n    "answer": "human" / "AI",  \n}\n```\n- You must not provide any other text, except JSON.\n- In the reason part, don\'t use symbol like " , ` , \\n, use only normal character.\nHere is C++ source code:\n\n\n#include <bits/stdc++.h>\nusing namespace std;\n\nint main() {\n    ios_base::sync_with_stdio(false);\n    cin.tie(nullptr);\n\n    int n, m;\n    cin >> n >>

## Deepseek

In [6]:
from openai import OpenAI
client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [7]:
BACKUP_PATH = "./Backups/Result-DeepseekR1-Reasoner.csv"

def save_responses(res):
    df = pd.DataFrame(res, columns=['back_up'])
    df.to_csv(BACKUP_PATH,index=False)

In [8]:
try:
    saved_df = pd.read_csv(BACKUP_PATH)
    responses = saved_df['back_up'].to_list()
except Exception as e:
    print(f"ERROR : {e}")
    responses = []
len(responses)

50

In [ ]:
err_logs = []
from rich import print

from IPython.display import clear_output
for index, row in test_df[len(responses):].iterrows():
  if index%50 == 0 and index != 0:
    save_responses(responses)
  source = row['source']
  print(f"ASKED : {index}")
  try:
    response = client.chat.completions.create(
      model="deepseek-reasoner",
      messages=[
          {"role": "system", "content": prompt},
          {"role": "user", "content": source},
      ],
      stream=False
    )
    clear_output()
    responses.append(response.choices[0].message.content)
    print(response.choices[0].message.content)
  except Exception as e:
    clear_output()
    err_logs.append((index,e))
    print(f"ERROR at {index} : {e}")

  

In [10]:
save_responses(responses)

In [12]:
import re
import json


result_n = len(responses)
answer_df = test_df.iloc[:result_n]
answer_df['answer_text'] = responses

C:\Users\Pakin\AppData\Local\Temp\ipykernel_5764\1922296617.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_text'] = responses


In [13]:
pattern = re.compile(r'```json\s*(\{.*?\})\s*```', re.DOTALL)
answer_df['answer_json_txt'] = answer_df['answer_text'].apply(lambda x: pattern.search(x).group(1) if pattern.search(x) else x)

C:\Users\Pakin\AppData\Local\Temp\ipykernel_5764\1101181029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_json_txt'] = answer_df['answer_text'].apply(lambda x: pattern.search(x).group(1) if pattern.search(x) else x)


In [14]:
answer_df['answer_dict'] = answer_df['answer_json_txt'].apply(json.loads)
answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
answer_df['model_reason'] = answer_df['answer_dict'].apply(lambda x: x['reason'])
answer_df['verdict'] = answer_df['writer'] == answer_df['model_answer']
answer_df = answer_df[['writer','model_answer','verdict','model_reason','website','source']]

C:\Users\Pakin\AppData\Local\Temp\ipykernel_5764\1828042203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_dict'] = answer_df['answer_json_txt'].apply(json.loads)
C:\Users\Pakin\AppData\Local\Temp\ipykernel_5764\1828042203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
C:\Users\Pakin\AppData\Local\Temp\ipykernel_5764\1828042203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [15]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','Reason','Website']] = answer_df[['writer','model_answer','verdict','model_reason','website']]

In [16]:
# I forgot about upper-lower case T-T
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']

In [17]:
filtered_df

,Label,Model_Answer,Verdict,Reason,Website
0,AI,human,False,The code uses standard competitive programming...,https://codeforces.com
1,AI,human,False,The code uses competitive programming idioms l...,https://codeforces.com
2,AI,human,False,The code uses competitive programming practice...,https://programming.in.th
3,human,human,True,The code uses competitive programming shortcut...,https://codeforces.com
4,AI,human,False,The code uses competitive programming practice...,https://programming.in.th
...,...,...,...,...,...
662,AI,AI,True,The code uses an extremely inefficient method ...,https://programming.in.th
663,human,human,True,The code uses global variables and a recursive...,https://firefly.gchan.moe
664,human,human,True,The code includes unused macros like MAX and M...,https://codeforces.com
665,human,human,True,The code uses common competitive programming s...,https://firefly.gchan.moe


In [ ]:
filtered_df.to_csv("./Results/_2/Result-DeepseekR1-Reasoner.csv",index=False)

In [20]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.527736131934033